In [1]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import pandas as pd
import os
import cv2
import numpy as np
from streamlit_drawable_canvas import st_canvas


## Extract and Track Figure Pages

In [2]:
def extract_figure_pages_with_ocr(pdf_path, extracted_figures_path, year):
    # Use OCR to extract text
    ocr_config = r'--oem 3 --psm 6'
    doc = fitz.open(pdf_path)
    saved_figures = []
    figure_1_detected = False
    for page_num, page in enumerate(doc):
        # Increase DPI of the image
        zoom_x = 2 # horizontal zoom
        zoom_y = 2  # vertical zoom
        mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension

        # Render the page into an image using the zoom matrix
        pix = page.get_pixmap(matrix=mat)

        # Open the image with PIL
        img_0 = Image.open(io.BytesIO(pix.tobytes("ppm")))
        # Rotate the image 90 degrees clockwise

        # Get text from images
        text_0 = pytesseract.image_to_string(img_0, config=ocr_config)

        saved = False
        rotation = 0
        ## to avoid overwriting figure 1 with figure 10, etc. 
        figure_range = range(1, 20)
        if(figure_1_detected):
            figure_range = range(2,19)

        for i in figure_range:
            figure_keywords = [f'Figure {i}'] ## keep as array in case needs to expand
            figure_name = f"F{i}_{os.path.splitext(os.path.basename(pdf_path))[0]}.png"
            if(any(substring in text_0 for substring in figure_keywords)):
                img_0.save(extracted_figures_path +"/" + str(year) + "/" + figure_name)
                saved = True
                if(i == 1):
                    figure_1_detected = True
                print(f"Saved {figure_name} from {pdf_path}")
                saved_figures.append({
                    "figure_name" : figure_name, 
                    "figure_number": i,
                    "page_number": page_num + 1,
                    "image_rotation": rotation
                })

        if not saved and (len(text_0) < 2000): 
            img_m_90 = img_0.rotate(-90, expand=True)  # Use expand=True to resize the image to fit the new orientation
            text_m_90 = pytesseract.image_to_string(img_m_90, config=ocr_config)
            for i in figure_range:
                figure_keywords = [f'Figure {i}'] ## keep as array in case needs to expand
                figure_name = f"F{i}_{os.path.splitext(os.path.basename(pdf_path))[0]}.png"
                rotation = 0
                if(any(substring in text_m_90 for substring in figure_keywords)):
                    img_m_90.save(extracted_figures_path +"/" + str(year) + "/" + figure_name)
                    saved = True
                    rotation = -90
                    if(i == 1):
                        figure_1_detected = True
                    print(f"Saved {figure_name} from {pdf_path}")
                    saved_figures.append({
                        "figure_name" : figure_name, 
                        "figure_number": i,
                        "page_number": page_num + 1,
                        "image_rotation": rotation
                    })

            if (len(text_m_90) < 2000): 
                img_p_90 = img_0.rotate(90, expand=True)  # Use expand=True to resize the image to fit the new orientation
                text_p_90 = pytesseract.image_to_string(img_p_90, config=ocr_config)
                for i in figure_range:
                    figure_keywords = [f'Figure {i}'] ## keep as array in case needs to expand
                    figure_name = f"F{i}_{os.path.splitext(os.path.basename(pdf_path))[0]}.png"
                    rotation = 0
                    if(any(substring in text_p_90 for substring in figure_keywords)):
                        img_p_90.save(extracted_figures_path +"/" + str(year) + "/" + figure_name)
                        saved = True
                        rotation = 90 
                        if(i == 1):
                            figure_1_detected = True
                        print(f"Saved {figure_name} from {pdf_path}")
                        saved_figures.append({
                            "figure_name" : figure_name, 
                            "figure_number": i,
                            "page_number": page_num + 1,
                            "image_rotation": rotation
                        })
        
    return saved_figures


In [3]:

# Define the directory to process
extracted_figures_dir = '../extracted_figure_pages'

# List to hold data for the DataFrame
figure_data = []
papers_with_no_identified_figures = []


for year in range(1985,2024): 
    paper_year_dir = f'../papers_by_year/{year}'
    figure_year_dir = f'{extracted_figures_dir}/{year}'
    # Loop through all files in the directory
    test_files =  [
            # "../papers_by_year/2010/Azoulay et al._2010_ASQ_Nasty Brutish and Short - Embeddedness Failure in the Pharma Industry.pdf", 
            # "../papers_by_year/2010/Bresman_2010_OrgSci_Explaining Employee Engagement with Strategic Change.pdf"
            ]
    
    for paper_file in os.listdir(paper_year_dir):
        paper_path = os.path.join(paper_year_dir, paper_file)

        ## make year directory in extracted figures folder if needed
        os.makedirs(figure_year_dir, exist_ok=True)

        if os.path.isfile(paper_path) and paper_path not in test_files:
            print(f'processing {paper_path}')
            figures =  extract_figure_pages_with_ocr(paper_path, extracted_figures_dir, year)
            # Process each figure
            for figure_info in figures:
                figure_data.append({
                    'original paper': paper_file,
                    'figure name': figure_info["figure_name"],
                    'figure number': figure_info["figure_number"], 
                    'year': year, 
                    'page number': figure_info["page_number"],
                    'image rotation': figure_info["image_rotation"]
                })

            if(len(figures) == 0):
                papers_with_no_identified_figures.append({
                    'original paper': paper_file,
                    'year': year
                })




processing ../papers_by_year/1985/Miller & Evko_1985_HR_An Ethnographic Study of the Influence of a Mobile Home Community.pdf
processing ../papers_by_year/1992/Attewall_1992_OrgSci_Technology Diffusion and Organizatiional Learning.pdf
processing ../papers_by_year/1994/Phillips_1994_OrgSci_Industry Mindsets - Exploring the Cultures of two macro organizational settings.pdf
processing ../papers_by_year/1994/Alvesson_1994_OrgStudies_Talking in Organizations.pdf
processing ../papers_by_year/1995/Hall & Noguchi_1995_HR_Engaging in Kenson.pdf
processing ../papers_by_year/1996/Kayes_2004_HR_The 1996 Mount Everest climbing disaster.pdf
Saved F1_Kayes_2004_HR_The 1996 Mount Everest climbing disaster.png from ../papers_by_year/1996/Kayes_2004_HR_The 1996 Mount Everest climbing disaster.pdf
processing ../papers_by_year/1998/Fineman_1998_OrgStudies_Street Level Bureacrats and The social construction of environmental control.pdf
processing ../papers_by_year/2000/Human et al_2000_ASQ_Legitimacy Build

KeyboardInterrupt: 

In [5]:
# Create DataFrames
figures_df = pd.DataFrame(figure_data, columns=['original paper', 'figure name', 'figure number', 'year', 'page number', 'image rotation'])
no_figures_df = pd.DataFrame(papers_with_no_identified_figures, columns=['original paper', 'year'])
figures_df.head()

,original paper,figure name,figure number,year,page number,image rotation
0,Kayes_2004_HR_The 1996 Mount Everest climbing ...,F1_Kayes_2004_HR_The 1996 Mount Everest climbi...,1,1996,17,0
1,Glynn_2000_OrgSci_When Cymbals become Symbols.pdf,F1_Glynn_2000_OrgSci_When Cymbals become Symbo...,1,2000,10,0
2,James_2000_OrgSci_Race-Related Differences_Qua...,F1_James_2000_OrgSci_Race-Related Differences_...,1,2000,7,0
3,James_2000_OrgSci_Race-Related Differences_Qua...,F2_James_2000_OrgSci_Race-Related Differences_...,2,2000,11,0
4,Bansal & Roth_2000_AMJ_Why Companies Go Green.pdf,F1_Bansal & Roth_2000_AMJ_Why Companies Go Gre...,1,2000,2,0


In [6]:
# Define the path to the Excel file
figure_tracking_file_path = "../extracted_figure_pages.xlsx"

# Check if the file exists
if os.path.exists(figure_tracking_file_path):
    # Read the existing Excel file
    existing_df = pd.read_excel(figure_tracking_file_path)
    # Concatenate existing and new data
    combined_df = pd.concat([existing_df, figures_df], ignore_index=True)
#     # Remove duplicates. Adjust the subset according to what uniquely identifies a row
    combined_df.drop_duplicates(subset=['original paper', 'figure name', 'figure number', 'year', 'page number', 'image rotation'], inplace=True)
else:
    combined_df = figures_df

# Sort by 'year'
combined_df.sort_values(by='year', inplace=True)

# Save to Excel
combined_df.to_excel(figure_tracking_file_path, index=False)

In [7]:
no_figures_df

,original paper,year
0,Miller & Evko_1985_HR_An Ethnographic Study of...,1985
1,Attewall_1992_OrgSci_Technology Diffusion and ...,1992
2,Phillips_1994_OrgSci_Industry Mindsets - Explo...,1994
3,Alvesson_1994_OrgStudies_Talking in Organizati...,1994
4,Hall & Noguchi_1995_HR_Engaging in Kenson.pdf,1995
...,...,...
199,Burchiellaro_2021_OrgStudies_Queering Control ...,2021
200,"Brooks, Grgulis & Cook_2021_HR_Unlearning and ...",2021
201,Morlacchi_2021_OrgStudies_The Performative Pow...,2021
202,"Garcia-Lorenzo, Sell-Trujillo & Donnelly_2021_...",2021


In [8]:
# Define the path to the Excel file for no-figures papers
no_figures_file_path = "../no_extracted_figure_papers.xlsx"

# Check if the file exists
if os.path.exists(no_figures_file_path):
    # Read the existing Excel file
    existing_no_figures_df = pd.read_excel(no_figures_file_path)
    # Concatenate existing and new data
    combined_no_figures_df = pd.concat([existing_no_figures_df, no_figures_df], ignore_index=True)
    # Remove duplicates. Adjust the subset according to what uniquely identifies a row
    combined_no_figures_df.drop_duplicates(subset=['original paper', 'year'], inplace=True)
else:
    combined_no_figures_df = no_figures_df

# Sort by 'year'
combined_no_figures_df.sort_values(by='year', inplace=True)

# Save to Excel
combined_no_figures_df.to_excel(no_figures_file_path, index=False)

print("No-figure paper data saved successfully to Excel.")

No-figure paper data saved successfully to Excel.
